# f) Ngonye Falls Power and Energy Periods
Produces summary, agrregated results from the daily power and energy modelling.


## Inputs

| Data                       | Source                                        | Description                                 |
|----------------------------|-----------------------------------------------|---------------------------------------------|
| [Model Name]_pe_daily.csv | Notebook: e_power_energy | Daily power output  |


## Outputs
| File                           | Description                                 |
|--------------------------------|---------------------------------------------|
|    [Model Name]_pe_daily.csv ||
|    [Model Name]_pe_monthly.csv||
|    [Model Name]_pe_yearly.csv||
|    [Model Name]_pe_percentiles.csv||
|    [Model Name]_pe_monthly_day.csv||
|    [Model Name]_pe_daily_slim.csv||
|    [Model Name]_pe_weekly.csv||
|    [Model Name]_pe_weekly_slim.csv||
|    [Model Name]_pe_calmonthly.csv||
|    [Model Name]_pe_fdc.csv||
|    [Model Name]_pe_power_exceed.csv||
|    models.csv| |

## Libraries and Setup

In [8]:
import numpy as np
import pandas as pd
from scipy import interpolate
import math
import statsmodels.tsa.stattools as statools

## Parameters

In [9]:
input_data='./input_data/'
#output_data='./output_data/2020/'
output_data='./output_data/'

In [10]:
model_name='Base Case'#MW153#MW144 'MW162'#'Fixed Tailwater'#'Base Case' Headpond50cm EWRAllCs
dryrun=False #Dont write output files

In [11]:
#models=pd.read_csv(input_data + "models.csv").set_index('ModelName')
models=pd.read_csv(input_data + "2020/models.csv").set_index('ModelName')
model=models.loc[model_name]

out_prefix=model['OutputPrefix']#'base'
if out_prefix!='base':
    out_prefix="scenarios/" + out_prefix
    
plant_capacity=model['Capacity']#180
canal_capacity=plant_capacity/180*1100#1100

model

OutputPrefix                       base
Description                   Base Case
EWRCatSet              Recommendation 1
HeadpondLift                          0
Capacity                            180
FixedTailwater                        0
Mean                             838464
P95                              719835
P90                              758689
P75                              797362
P50                              843164
P25                              881920
P10                              928448
P5                               964978
StDev                           81402.6
CoefVar                       0.0970854
Max                         1.02646e+06
Min                              508216
CapFactor                      0.531751
CalYearMean                      838630
CalYearStDev                    77041.1
CalYearCoefVar                0.0918654
AutoCorrel                          NaN
Power_max                       194.454
CapFactor_Power_max            0.492323


## Load Daily Data

Load the daily time series which includes the exceedance values used for calculating daily EWRs.

In [12]:
daily=pd.read_csv(output_data + 'ngonye_daily' + '_base_pe.csv') #ngonye_daily_WC1_1_base_pe
daily=daily.set_index(pd.to_datetime(daily['Date'],format="%Y-%m-%d"))
daily=daily.drop('Date',axis=1)
daily.tail(5)

,LaggedDate,VicFalls,Conversion,Flow,Exceedance,Year,Month,Day,MonthId,WaterYear,...,EffGen4,PowerTurbine1,PowerTurbine2,PowerTurbine3,PowerTurbine4,Power,Energy,PerfMwPerCumec,LowHeadShutoff,LowFlowShutoff
Date,,,,,,,,,,,,,,,,,,,,,
2017-09-26,2017-10-07,204.0855,0.982689,200.552607,0.955,2017,9,26,2017.09,2016,...,NaN,23.062116,NaN,NaN,NaN,23.062116,553.490790,0.114993,0,0
2017-09-27,2017-10-08,204.0855,0.982689,200.552607,0.955,2017,9,27,2017.09,2016,...,NaN,23.062116,NaN,NaN,NaN,23.062116,553.490790,0.114993,0,0
2017-09-28,2017-10-09,204.0855,0.982689,200.552607,0.955,2017,9,28,2017.09,2016,...,NaN,23.062116,NaN,NaN,NaN,23.062116,553.490790,0.114993,0,0
2017-09-29,2017-10-10,204.0855,0.982689,200.552607,0.955,2017,9,29,2017.09,2016,...,NaN,23.062116,NaN,NaN,NaN,23.062116,553.490790,0.114993,0,0
2017-09-30,2017-10-11,200.9197,0.995488,200.013130,0.958,2017,9,30,2017.09,2016,...,NaN,22.949432,NaN,NaN,NaN,22.949432,550.786366,0.114740,0,0


In [13]:
daily=daily.drop(daily.loc['2020-05-01':].index)
daily.tail(5)

,LaggedDate,VicFalls,Conversion,Flow,Exceedance,Year,Month,Day,MonthId,WaterYear,...,EffGen4,PowerTurbine1,PowerTurbine2,PowerTurbine3,PowerTurbine4,Power,Energy,PerfMwPerCumec,LowHeadShutoff,LowFlowShutoff
Date,,,,,,,,,,,,,,,,,,,,,
2017-09-26,2017-10-07,204.0855,0.982689,200.552607,0.955,2017,9,26,2017.09,2016,...,NaN,23.062116,NaN,NaN,NaN,23.062116,553.490790,0.114993,0,0
2017-09-27,2017-10-08,204.0855,0.982689,200.552607,0.955,2017,9,27,2017.09,2016,...,NaN,23.062116,NaN,NaN,NaN,23.062116,553.490790,0.114993,0,0
2017-09-28,2017-10-09,204.0855,0.982689,200.552607,0.955,2017,9,28,2017.09,2016,...,NaN,23.062116,NaN,NaN,NaN,23.062116,553.490790,0.114993,0,0
2017-09-29,2017-10-10,204.0855,0.982689,200.552607,0.955,2017,9,29,2017.09,2016,...,NaN,23.062116,NaN,NaN,NaN,23.062116,553.490790,0.114993,0,0
2017-09-30,2017-10-11,200.9197,0.995488,200.013130,0.958,2017,9,30,2017.09,2016,...,NaN,22.949432,NaN,NaN,NaN,22.949432,550.786366,0.114740,0,0


## Monthly


In [14]:

monthly=daily.groupby("MonthId").mean().drop(['Day','EWRBandNo','EWRRefExceedance','Energy'],axis=1)

if 'VicFalls' in monthly.columns:
    monthly=monthly.drop(['VicFalls','Conversion','Volume','WaterDay','WaterWeek','Volume'],axis=1)

monthly=monthly.join(daily.groupby("MonthId").agg(    
   Power_max=('Power', 'max'),
   Power_min=('Power', 'min'),   
   Turbines_max=('Turbines', 'max'),    
   Turbines_min=('Turbines', 'min'),   
   Energy=('Energy', 'sum'),
))
monthly['Days']=monthly.apply(lambda x: pd.Period(year=x['Year'],month=x['Month'],freq='D').days_in_month,axis=1)
monthly['CapFactor']=monthly['Energy']/(plant_capacity*24*monthly['Days'])

monthly.tail(12)

,Flow,Exceedance,Year,Month,WaterYear,WaterMonth,Flow_difference,Flow_difference_pct,EWRChannelA,EWRChannelC,...,PerfMwPerCumec,LowHeadShutoff,LowFlowShutoff,Power_max,Power_min,Turbines_max,Turbines_min,Energy,Days,CapFactor
MonthId,,,,,,,,,,,,,,,,,,,,,
2016.10,186.893688,0.973613,2016.0,10.0,2016.0,1.0,1.545550,0.008254,31.000000,4.000000,...,0.130904,0.0,0.0,29.656737,19.751596,1,1,18254.782857,31,0.136311
2016.11,186.892986,0.969233,2016.0,11.0,2016.0,2.0,3.153585,0.016633,25.000000,4.000000,...,0.150639,0.0,0.0,39.045483,18.622058,1,1,20517.729239,30,0.158316
2016.12,369.415926,0.701097,2016.0,12.0,2016.0,3.0,16.532126,0.041183,29.387097,4.677419,...,0.174440,0.0,0.0,106.590505,39.225224,2,1,48084.856271,31,0.359057
2017.01,851.174020,0.394677,2017.0,1.0,2016.0,4.0,26.578244,0.034651,44.387097,6.677419,...,0.162207,0.0,0.0,176.969980,67.378470,4,2,103391.402018,31,0.772039
2017.02,1237.295368,0.273750,2017.0,2.0,2016.0,5.0,31.877670,0.023640,49.000000,8.000000,...,0.147978,0.0,0.0,192.177553,171.445726,4,4,121669.763532,28,1.005868
2017.03,2422.303967,0.145806,2017.0,3.0,2016.0,6.0,75.443535,0.029406,49.000000,9.000000,...,0.074986,0.0,0.0,181.288164,117.641823,4,4,115493.952639,31,0.862410
2017.04,3418.418249,0.063667,2017.0,4.0,2016.0,7.0,36.866143,0.011125,70.000000,14.000000,...,0.038045,0.0,0.0,142.741576,116.923599,4,4,92099.952249,30,0.710648
2017.05,2195.626922,0.156581,2017.0,5.0,2016.0,8.0,38.011322,0.017650,55.000000,13.000000,...,0.075317,0.0,0.0,177.259550,143.903364,4,4,119278.435042,31,0.890669
2017.06,1118.647129,0.312300,2017.0,6.0,2016.0,9.0,32.020074,0.029916,53.000000,12.000000,...,0.141569,0.0,0.0,188.994148,96.505982,4,2,112359.569599,30,0.866972


## Yearly

In [15]:
yearly=daily.groupby("WaterYear").agg(
   Flow_max=('Flow', 'max'),
   Flow_min=('Flow', 'min'), 
   EWRProportion_max=('EWRProportion', 'max'),
   EWRProportion_min=('EWRProportion', 'min'),
   HeadTurbine1_max=('HeadTurbine1', 'max'),
   HeadTurbine1_min=('HeadTurbine1', 'min'),    
   LevelTailwater_max=('LevelTailwater', 'max'),
   LevelTailwater_min=('LevelTailwater', 'min'),  
   Power_max=('Power', 'max'),
   Power_min=('Power', 'min'),    
   Energy=('Energy', 'sum'),
)
yearly['CapFactor']=yearly['Energy']/(plant_capacity*24*365)


In [16]:
yearly=yearly.drop(yearly.loc['2019':].index)
yearly.tail(5)

,Flow_max,Flow_min,EWRProportion_max,EWRProportion_min,HeadTurbine1_max,HeadTurbine1_min,LevelTailwater_max,LevelTailwater_min,Power_max,Power_min,Energy,CapFactor
WaterYear,,,,,,,,,,,,
2012,3614.802471,223.273423,0.508220,0.055826,25.083495,13.168827,974.894271,964.716509,191.310490,24.528540,843130.264390,0.534710
2013,3574.946119,195.061590,0.497279,0.056428,25.436464,13.247224,974.815874,964.405522,191.927167,20.671377,926459.111352,0.587557
2014,1460.253290,208.780681,0.546606,0.099878,25.098362,18.623511,969.484512,964.706890,192.407900,25.316443,884558.950509,0.560984
2015,2975.524395,176.218173,0.496598,0.067347,25.390405,14.500960,973.562138,964.405522,191.268651,21.964965,786286.526245,0.498660
2016,3889.991232,156.550907,0.487279,0.047442,25.390405,12.642403,975.420695,964.405522,192.177553,18.622058,851845.332264,0.540237


In [17]:
yearly['Prop4Units']=daily[['WaterYear','Turbines']].groupby('WaterYear').agg(lambda x: sum(np.where(x==4,1,0))/365)
yearly['Prop3Units']=daily[['WaterYear','Turbines']].groupby('WaterYear').agg(lambda x: sum(np.where(x==3,1,0))/365)
yearly['Prop2Units']=daily[['WaterYear','Turbines']].groupby('WaterYear').agg(lambda x: sum(np.where(x==2,1,0))/365)
yearly['Prop1Units']=daily[['WaterYear','Turbines']].groupby('WaterYear').agg(lambda x: sum(np.where(x==1,1,0))/365)
yearly['Prop0Units']=daily[['WaterYear','Turbines']].groupby('WaterYear').agg(lambda x: sum(np.where(x==0,1,0))/365)
yearly['Days0Units']=daily[['WaterYear','Turbines']].groupby('WaterYear').agg(lambda x: sum(np.where(x==0,1,0)))


## Power and energy exceedance

In [18]:
power_exceed=pd.DataFrame(index=np.arange(0,1.05,0.05))
power_exceed.index=power_exceed.index.rename('Exceedance')
power_exceed['Power']=np.percentile(daily['Power'],(1-power_exceed.index)*100)
power_exceed.head(3)

,Power
Exceedance,
0.00,193.908405
0.05,181.679247
0.10,173.476231


In [19]:
edc=pd.DataFrame(index=np.arange(0,1.05,0.05))
edc['Energy']=np.percentile(yearly['Energy'],(1-edc.index)*100)

edc.head(3)

,Energy
0.00,1.018971e+06
0.05,9.582529e+05
0.10,9.219147e+05


In [20]:
yearly['Exceedance']=pd.merge_asof(yearly.reset_index().sort_values('Energy'),edc.reset_index().sort_values('Energy'),left_on='Energy',right_on='Energy').set_index('WaterYear')['index']

## Calendar years
Jan-Dec rather than Oct-Sept

In [21]:
min_year=daily['Year'].min()
max_year=daily['Year'].max()


In [22]:
calyearly=daily.groupby("Year").agg(  
   Energy=('Energy', 'sum'),
).drop([min_year,max_year],axis=0)
calyearly['CapFactor']=calyearly['Energy']/(180*24*365)
calyearly.head(5)

,Energy,CapFactor
Year,,
1925,854344.223735,0.541822
1926,779350.985117,0.494261
1927,854874.185213,0.542158
1928,829351.404158,0.525971
1929,753745.429409,0.478022


## Monthly Day
15th of every month

In [23]:
monthly_day=daily.loc[daily['Day']==15]

## Daily Slim
Slim has less columns and values are rounded so file size is reduced

In [24]:
daily_slim=daily.loc[:,
                     ['Year',
                      'Month',
                      'Day',
                      'WaterYear',
                      'Flow',
                      'EWRRefExceedance',
                      'EWRBandNo',
                      'EWRChannelA',
                      'EWRChannelC',
                      'EWRChannelD',
                      'EWRChannelE',
                      'EWRChannelFG',
                      'EWRTotal',
                      'EWRProportion',
                      'FlowCanal',
                      'FlowSpill',
                      'SpillChannelA',
                      'SpillChannelC',
                      'SpillChannelD',
                      'SpillChannelE',
                      'SpillChannelFG',
                      'FlowChannelA',
                      'FlowChannelC',
                      'FlowChannelD',
                      'FlowChannelE',
                      'FlowChannelFG',
                      'FlowLeftChannel',
                      'FlowTurbine1',
                      'FlowTurbine2',
                      'FlowTurbine3',
                      'FlowTurbine4',
                      'LevelHeadpond',
                      'LevelTailwater',
                      'HeadlossLeftChannel',
                      'HeadlossCanal',
                      'LevelForebay',
                      'Turbines',
                      'HeadTurbine1',
                      'HeadTurbine2',
                      'HeadTurbine3',
                      'HeadTurbine4',
                      'EffTurbine1',
                      'EffTurbine2',
                      'EffTurbine3',
                      'EffTurbine4',
                      'PowerTurbine1',
                      'PowerTurbine2',
                      'PowerTurbine3',
                      'PowerTurbine4',
                      'Power',
                      'Energy'
                     ]
                    ].fillna(0).astype({
                        'Flow': 'int32',
                        'FlowCanal': 'int32',
                        'FlowSpill': 'int32',
                        'EWRChannelA': 'int32',
                        'EWRChannelC': 'int32',
                        'EWRChannelD': 'int32',    
                        'EWRChannelE': 'int32',
                        'EWRChannelFG': 'int32',
                        'EWRTotal': 'int32',    
                        'SpillChannelA': 'int32',
                        'SpillChannelC': 'int32',
                        'SpillChannelD': 'int32',    
                        'SpillChannelE': 'int32',
                        'SpillChannelFG': 'int32',
                        'FlowChannelA': 'int32',
                        'FlowChannelC': 'int32',
                        'FlowChannelD': 'int32',    
                        'FlowChannelE': 'int32',
                        'FlowChannelFG': 'int32',
                        'FlowLeftChannel': 'int32',
                        'FlowTurbine1': 'int32',
                        'FlowTurbine2': 'int32',
                        'FlowTurbine3': 'int32',
                        'FlowTurbine4': 'int32',
                        'Energy': 'int32',
                        
                    }).round({
                            'EWRProportion': 2,
                            'LevelHeadpond': 1,
                            'LevelTailwater': 1,
                            'HeadlossLeftChannel': 2,
                            'HeadlossCanal': 2,
                            'LevelForebay': 1,
                            'PowerTurbine1': 1,
                            'PowerTurbine2': 1,
                            'PowerTurbine3': 1,
                            'PowerTurbine4': 1,
                            'Power': 1,
                            'HeadTurbine1':1,
                            'HeadTurbine2':1,
                            'HeadTurbine3':1,
                            'HeadTurbine4':1,
                            'EffTurbine1':2,
                            'EffTurbine2':2,
                            'EffTurbine3':2,
                            'EffTurbine4':2,    
                    })
daily_slim.tail(12)

,Year,Month,Day,WaterYear,Flow,EWRRefExceedance,EWRBandNo,EWRChannelA,EWRChannelC,EWRChannelD,...,EffTurbine1,EffTurbine2,EffTurbine3,EffTurbine4,PowerTurbine1,PowerTurbine2,PowerTurbine3,PowerTurbine4,Power,Energy
Date,,,,,,,,,,,,,,,,,,,,,
2017-09-19,2017,9,19,2016,216,0.80,4,34,6,4,...,0.91,0.0,0.0,0.0,27.3,0.0,0.0,0.0,27.3,654
2017-09-20,2017,9,20,2016,213,0.80,4,34,6,4,...,0.91,0.0,0.0,0.0,26.6,0.0,0.0,0.0,26.6,638
2017-09-21,2017,9,21,2016,213,0.82,4,34,6,4,...,0.91,0.0,0.0,0.0,26.6,0.0,0.0,0.0,26.6,638
2017-09-22,2017,9,22,2016,209,0.82,4,34,6,4,...,0.90,0.0,0.0,0.0,25.5,0.0,0.0,0.0,25.5,612
2017-09-23,2017,9,23,2016,203,0.82,4,34,6,4,...,0.90,0.0,0.0,0.0,23.8,0.0,0.0,0.0,23.8,572
2017-09-24,2017,9,24,2016,200,0.82,4,34,6,4,...,0.89,0.0,0.0,0.0,23.1,0.0,0.0,0.0,23.1,553
2017-09-25,2017,9,25,2016,200,0.82,4,34,6,4,...,0.89,0.0,0.0,0.0,23.1,0.0,0.0,0.0,23.1,553
2017-09-26,2017,9,26,2016,200,0.82,4,34,6,4,...,0.89,0.0,0.0,0.0,23.1,0.0,0.0,0.0,23.1,553
2017-09-27,2017,9,27,2016,200,0.82,4,34,6,4,...,0.89,0.0,0.0,0.0,23.1,0.0,0.0,0.0,23.1,553


## Weekly & Weekly Slim
Slim has less columns and values are rounded so file size is reduced

In [25]:
weekly=daily.groupby(["WaterYear","WaterWeek"]).mean().drop(['Month','WaterMonth','Year','MonthId','Day','WaterDay','EWRBandNo','EWRRefExceedance','Turbines','Energy'],axis=1)

#if 'VicFalls' in weekly.columns:
weekly=weekly.drop(['VicFalls','Conversion','Volume'],axis=1)

weekly=weekly.join(daily.groupby(["WaterYear","WaterWeek"]).agg(    
   Flow_max=('Flow', 'max'),
   Flow_min=('Flow', 'min'), 
   EWRProportion_max=('EWRProportion', 'max'),
   EWRProportion_min=('EWRProportion', 'min'),
   HeadTurbine1_max=('HeadTurbine1', 'max'),
   HeadTurbine1_min=('HeadTurbine1', 'min'),    
   LevelTailwater_max=('LevelTailwater', 'max'),
   LevelTailwater_min=('LevelTailwater', 'min'),  
   Power_max=('Power', 'max'),
   Power_min=('Power', 'min'),    
   Energy=('Energy', 'sum'),
))
weekly=weekly.join(daily[["WaterYear","WaterWeek","Flow"]].groupby(["WaterYear","WaterWeek"]).count().rename(columns={'Flow':'Count'}))
weekly['CapFactor']=weekly['Energy']/(180*24*weekly['Count'])
weekly=weekly.join(daily.reset_index('Date')[["WaterYear","WaterWeek","Date"]].groupby(["WaterYear","WaterWeek"]).first().rename(columns={'Date':'StartDate'}))

weekly.tail(12)

Flow  Exceedance  Flow_difference  \
WaterYear WaterWeek                                            
2016      42         451.943382    0.593143         6.536288   
          43         411.172524    0.638714         4.724796   
          44         377.834520    0.678857         5.261728   
          45         344.446201    0.716714         4.257422   
          46         322.314771    0.751857         2.882230   
          47         296.218764    0.791143         3.371720   
          48         271.256811    0.834000         3.832214   
          49         250.464258    0.873286         3.094980   
          50         226.821410    0.915571         2.855981   
          51         215.657828    0.933429         1.480060   
          52         200.905599    0.953571         1.303758   
          53         200.013130    0.958000         0.539477   

                     Flow_difference_pct  EWRChannelA  EWRChannelC  \
WaterYear WaterWeek                                                  
2016      42                    0.014538    50.000000    10.000000   
          43                    0.011456    50.000000    10.000000   
          44                    0.013976    43.714286     8.285714   
          45                    0.012366    39.000000     7.000000   
          46                    0.009020    39.000000     7.000000   
          47                    0.011315    39.000000     7.000000   
          48                    0.014055    38.285714     6.857143   
          49                    0.012470    34.000000     6.000000   
          50                    0.012523    34.000000     6.000000   
          51                    0.006946    34.000000     6.000000   
          52                    0.006443    34.000000     6.000000   
          53                    0.002697    34.000000     6.000000   

                     EWRChannelD  EWRChannelE  EWRChannelFG    EWRTotal  ...  \
WaterYear WaterWeek                                                      ...   
2016      42            9.000000     9.000000    108.000000  186.000000  ...   
          43            9.000000     9.000000    108.000000  186.000000  ...   
          44            6.714286     6.142857     80.571429  145.428571  ...   
          45            5.000000     4.000000     60.000000  115.000000  ...   
          46            5.000000     4.000000     60.000000  115.000000  ...   
          47            5.000000     4.000000     60.000000  115.000000  ...   
          48            4.857143     3.857143     57.714286  111.571429  ...   
          49            4.000000     3.000000     44.000000   91.000000  ...   
          50            4.000000     3.000000     44.000000   91.000000  ...   
          51            4.000000     3.000000     44.000000   91.000000  ...   
          52            4.000000     3.000000     44.000000   91.000000  ...   
          53            4.000000     3.000000     44.000000   91.000000  ...   

                     HeadTurbine1_max  HeadTurbine1_min  LevelTailwater_max  \
WaterYear WaterWeek                                                           
2016      42                24.149540         23.875651          965.589031   
          43                24.268319         24.044971          965.380858   
          44                24.397504         24.164017          965.236958   
          45                24.520135         24.355542          965.083218   
          46                24.660401         24.535417          964.993439   
          47                24.807667         24.704049          964.908128   
          48                24.940660         24.836336          964.840915   
          49                24.992931         24.902176          964.778028   
          50                25.067056         25.014660          964.730913   
          51                25.096172         25.067056          964.713474   
          52                25.115752         25.111050          964.706442   
 

In [26]:
weekly_slim=weekly.loc[:,
                     [
                      'StartDate',
                      'Flow',
                      'EWRChannelA',
                      'EWRChannelC',
                      'EWRChannelD',
                      'EWRChannelE',
                      'EWRChannelFG',
                      'EWRTotal',
                      'EWRProportion',
                      'FlowCanal',
                      'FlowSpill',
                      'SpillChannelA',
                      'SpillChannelC',
                      'SpillChannelD',
                      'SpillChannelE',
                      'SpillChannelFG',
                      'FlowChannelA',
                      'FlowChannelC',
                      'FlowChannelD',
                      'FlowChannelE',
                      'FlowChannelFG',
                      'FlowLeftChannel',
                      'FlowTurbine1',
                      'FlowTurbine2',
                      'FlowTurbine3',
                      'FlowTurbine4',
                      'LevelHeadpond',
                      'LevelTailwater',
                      'HeadlossLeftChannel',
                      'HeadlossCanal',
                      'LevelForebay',
                      'HeadTurbine1',
                      'HeadTurbine2',
                      'HeadTurbine3',
                      'HeadTurbine4',
                      'EffTurbine1',
                      'EffTurbine2',
                      'EffTurbine3',
                      'EffTurbine4',
                      'PowerTurbine1',
                      'PowerTurbine2',
                      'PowerTurbine3',
                      'PowerTurbine4',
                      'Power',
                      'Energy'
                     ]
                    ].fillna(0).astype({
                        'Flow': 'int32',
                        'FlowCanal': 'int32',
                        'FlowSpill': 'int32',
                        'EWRChannelA': 'int32',
                        'EWRChannelC': 'int32',
                        'EWRChannelD': 'int32',    
                        'EWRChannelE': 'int32',
                        'EWRChannelFG': 'int32',
                        'EWRTotal': 'int32',
                        'SpillChannelA': 'int32',
                        'SpillChannelC': 'int32',
                        'SpillChannelD': 'int32',    
                        'SpillChannelE': 'int32',
                        'SpillChannelFG': 'int32',
                        'FlowChannelA': 'int32',
                        'FlowChannelC': 'int32',
                        'FlowChannelD': 'int32',    
                        'FlowChannelE': 'int32',
                        'FlowChannelFG': 'int32',
                        'FlowLeftChannel': 'int32',
                        'FlowTurbine1': 'int32',
                        'FlowTurbine2': 'int32',
                        'FlowTurbine3': 'int32',
                        'FlowTurbine4': 'int32',
                        'Energy': 'int32',
                        
                    }).round({
                            'EWRProportion':2,
                            'LevelHeadpond': 1,
                            'LevelTailwater': 1,
                            'HeadlossLeftChannel': 2,
                            'HeadlossCanal': 2,
                            'LevelForebay': 1,
                            'PowerTurbine1': 1,
                            'PowerTurbine2': 1,
                            'PowerTurbine3': 1,
                            'PowerTurbine4': 1,
                            'Power': 1,
                            'HeadTurbine1':1,
                            'HeadTurbine2':1,
                            'HeadTurbine3':1,
                            'HeadTurbine4':1,
                            'EffTurbine1':2,
                            'EffTurbine2':2,
                            'EffTurbine3':2,
                            'EffTurbine4':2,
                    })

## Calendar Months

In [27]:
calmonthly=pd.DataFrame({'WaterMonth': [1,2,3,4,5,6,7,8,9,10,11,12],'MonthName': ['Oct','Nov','Dec','Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep'], 'Month':[10,11,12,1,2,3,4,5,6,7,8,9]})
calmonthly=calmonthly.set_index('WaterMonth')
calmonthly=calmonthly.join(monthly.groupby(["WaterMonth"]).agg(    
   EWRProportion_max=('EWRProportion', 'max'),
   EWRProportion_mean=('EWRProportion', 'mean'),
   EWRProportion_median=('EWRProportion', 'median'),
   EWRProportion_min=('EWRProportion', 'min'),
    
    EWRTotal_max=('EWRTotal', 'max'),
   EWRTotal_mean=('EWRTotal', 'mean'),
   EWRTotal_median=('EWRTotal', 'median'),
   EWRTotal_min=('EWRTotal', 'min'),   
    
   FlowCanal_max=('FlowCanal', 'max'),
   FlowCanal_mean=('FlowCanal', 'mean'),
   FlowCanal_median=('FlowCanal', 'median'),
   FlowCanal_min=('FlowCanal', 'min'),     
    
   HeadTurbine1_max=('HeadTurbine1', 'max'),
   HeadTurbine1_mean=('HeadTurbine1', 'mean'),
   HeadTurbine1_median=('HeadTurbine1', 'median'),
   HeadTurbine1_min=('HeadTurbine1', 'min'), 

   LevelTailwater_max=('LevelTailwater', 'max'),
   LevelTailwater_mean=('LevelTailwater', 'mean'),    
   LevelTailwater_median=('LevelTailwater', 'median'),    
   LevelTailwater_min=('LevelTailwater', 'min'),  
   LevelHeadpond_max=('LevelHeadpond', 'max'),
   LevelHeadpond_mean=('LevelHeadpond', 'mean'),    
   LevelHeadpond_median=('LevelHeadpond', 'median'),    
   LevelHeadpond_min=('LevelHeadpond', 'min'),  
   Turbines_mean=('Turbines', 'mean'),      
   Power_max=('Power', 'max'),
   Power_mean=('Power', 'mean'),
   Power_median=('Power', 'median'),
   Power_min=('Power', 'min'),    
   Energy_max=('Energy', 'max'),
   Energy_mean=('Energy', 'mean'),
   Energy_median=('Energy', 'median'),    
   Energy_min=('Energy', 'min'),
))
calmonthly['Energy_P95']=monthly[['WaterMonth','Energy']].groupby('WaterMonth').quantile(0.05)
calmonthly['Energy_P90']=monthly[['WaterMonth','Energy']].groupby('WaterMonth').quantile(0.10)
calmonthly['Energy_P75']=monthly[['WaterMonth','Energy']].groupby('WaterMonth').quantile(0.25)
calmonthly['Energy_P25']=monthly[['WaterMonth','Energy']].groupby('WaterMonth').quantile(0.75)
calmonthly['Energy_P10']=monthly[['WaterMonth','Energy']].groupby('WaterMonth').quantile(0.90)
calmonthly['Energy_P5']=monthly[['WaterMonth','Energy']].groupby('WaterMonth').quantile(0.95)

calmonthly['Energy_stdev']=monthly[['WaterMonth','Energy']].groupby('WaterMonth').std()
calmonthly['Energy_coefvar']=calmonthly['Energy_stdev']/calmonthly['Energy_mean']

calmonthly

,MonthName,Month,EWRProportion_max,EWRProportion_mean,EWRProportion_median,EWRProportion_min,EWRTotal_max,EWRTotal_mean,EWRTotal_median,EWRTotal_min,...,Energy_median,Energy_min,Energy_P95,Energy_P90,Energy_P75,Energy_P25,Energy_P10,Energy_P5,Energy_stdev,Energy_coefvar
WaterMonth,,,,,,,,,,,,,,,,,,,,,
1,Oct,10,0.578553,0.396933,0.400655,0.274923,120.0,93.623309,97.000000,52.0,...,21318.693809,6668.218097,16473.492046,18232.212209,19304.884191,26708.536284,31466.869330,36586.724308,7042.680213,0.300787
2,Nov,11,0.403429,0.331518,0.334202,0.218705,114.0,89.358423,93.000000,50.0,...,26921.131804,15892.504150,19560.731929,21472.445920,23136.698850,32402.254913,41094.989133,45588.843595,8967.171547,0.306616
3,Dec,12,0.288584,0.241700,0.242757,0.157837,126.0,100.289629,104.000000,59.0,...,48084.856271,27971.791352,36645.012097,38258.282678,42951.712099,60742.125048,68583.791713,84571.288743,14425.402351,0.275372
4,Jan,1,0.222432,0.177347,0.181308,0.097665,150.0,117.763441,120.000000,73.0,...,76931.319991,42169.691465,56368.060034,62172.122010,69587.370766,97408.144526,117326.101019,120833.320530,20486.521889,0.244169
5,Feb,2,0.188832,0.129466,0.139340,0.023961,197.0,138.959744,137.000000,85.0,...,98603.438723,4972.912035,72086.597809,79205.320063,89244.275633,111754.519862,119378.596554,121412.882793,18662.406260,0.190080
6,Mar,3,0.182992,0.095462,0.091525,0.040246,315.0,194.510926,185.451613,107.0,...,111345.847995,0.000000,20050.327214,76462.033977,95087.752470,121680.078035,129854.881493,132843.577722,30306.066543,0.292099
7,Apr,4,0.165295,0.085109,0.082042,0.049993,364.0,231.265233,217.000000,125.0,...,100044.115710,0.000000,58607.459908,73218.225413,87273.472395,114982.549395,129301.212581,133642.959777,26305.577811,0.267540
8,May,5,0.223150,0.130801,0.128302,0.066393,340.0,265.271245,276.000000,147.0,...,116679.278672,44299.068846,87919.859424,91629.451965,106016.365248,126510.938718,131080.424050,133778.022259,16447.349603,0.144157
9,Jun,6,0.369425,0.200040,0.195288,0.108731,293.0,212.942652,210.000000,119.0,...,113936.512750,32726.734208,50690.979348,71790.780177,95352.726831,121524.842302,123474.809076,124749.699110,23185.985098,0.220721


## Flow duration curve

In [28]:
daily['ExceedanceRnd']=np.round(daily['Exceedance']*2,2)/2
fdc=daily.reset_index().groupby('ExceedanceRnd').mean().drop(['VicFalls','Conversion','Year','Month','Day','MonthId','WaterYear','WaterDay','WaterMonth','WaterWeek'],axis=1)
fdc

,Flow,Exceedance,Volume,Flow_difference,Flow_difference_pct,EWRRefExceedance,EWRBandNo,EWRChannelA,EWRChannelC,EWRChannelD,...,EffGen4,PowerTurbine1,PowerTurbine2,PowerTurbine3,PowerTurbine4,Power,Energy,PerfMwPerCumec,LowHeadShutoff,LowFlowShutoff
ExceedanceRnd,,,,,,,,,,,,,,,,,,,,,
0.000,8865.175953,0.001412,0.765951,137.615867,0.015867,0.026324,1.102941,65.735294,12.338235,14.691176,...,0.98,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.00000,0.0
0.005,6478.790089,0.004989,0.559767,115.269934,0.017456,0.041149,1.091954,72.436782,13.850575,17.701149,...,0.98,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.00000,0.0
0.010,5544.712707,0.010047,0.479063,68.659047,0.012349,0.077041,1.289941,72.680473,13.656805,17.674556,...,0.98,9.617783,9.617783,9.617783,9.617783,38.471131,923.307133,0.007106,0.56213,0.0
0.015,5190.964012,0.014953,0.448499,88.798283,0.017178,0.105349,1.401163,71.540698,12.941860,16.645349,...,0.98,22.887085,22.887085,22.887085,22.887085,91.548338,2197.160113,0.017651,0.00000,0.0
0.020,4777.593171,0.020076,0.412784,113.350030,0.023656,0.110465,1.406977,69.976744,13.156977,16.616279,...,0.98,24.716775,24.716775,24.716775,24.716775,98.867101,2372.810412,0.020720,0.00000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0.980,185.643897,0.979851,0.016040,2.397496,0.012948,0.897619,4.511905,28.017857,4.333333,2.708333,...,NaN,26.258804,NaN,NaN,NaN,26.258804,630.211305,0.141397,0.00000,0.0
0.985,175.260665,0.985178,0.015143,1.918049,0.010903,0.920385,4.682692,26.668269,4.254808,2.524038,...,NaN,24.754930,NaN,NaN,NaN,24.754930,594.118324,0.141300,0.00000,0.0
0.990,165.025295,0.990459,0.014258,2.091723,0.012641,0.944222,4.859259,25.370370,4.251852,2.325926,...,NaN,23.264526,NaN,NaN,NaN,23.264526,558.348614,0.140952,0.00000,0.0


## Add overall summary data to the model file

In [29]:

models.at[model_name,'Mean']=yearly['Energy'].mean()
models.at[model_name,'Max']=yearly['Energy'].max()
models.at[model_name,'Min']=yearly['Energy'].min()

models.at[model_name,'P95']=yearly['Energy'].quantile(0.05)
models.at[model_name,'P90']=yearly['Energy'].quantile(0.10)
models.at[model_name,'P75']=yearly['Energy'].quantile(0.25)
models.at[model_name,'P50']=yearly['Energy'].quantile(0.5)
models.at[model_name,'P25']=yearly['Energy'].quantile(0.75)
models.at[model_name,'P10']=yearly['Energy'].quantile(0.90)
models.at[model_name,'P5']=yearly['Energy'].quantile(0.95)

models.at[model_name,'StDev']=yearly['Energy'].std()
models.at[model_name,'CoefVar']=models.at[model_name,'StDev']/models.at[model_name,'Mean']

models.at[model_name,'CapFactor']=yearly['CapFactor'].mean()

models.at[model_name,'CalYearMean']=calyearly['Energy'].mean()
models.at[model_name,'CalYearStDev']=calyearly['Energy'].std()
models.at[model_name,'CalYearCoefVar']=models.at[model_name,'CalYearStDev']/models.at[model_name,'CalYearMean']

models.at[model_name,'Power_max']=yearly['Power_max'].max()
models.at[model_name,'CapFactor_Power_max']=calyearly['Energy'].mean()/(yearly['Power_max'].max()*24*365)

models.loc[model_name]



OutputPrefix                       base
Description                   Base Case
EWRCatSet              Recommendation 1
HeadpondLift                          0
Capacity                            180
FixedTailwater                        0
Mean                             832484
P95                              718504
P90                              756610
P75                              792216
P50                              838165
P25                              876248
P10                              921915
P5                               958253
StDev                           81971.2
CoefVar                       0.0984658
Max                         1.01897e+06
Min                              493939
CapFactor                      0.527958
CalYearMean                      832587
CalYearStDev                    78237.2
CalYearCoefVar                0.0939688
AutoCorrel                          NaN
Power_max                       193.908
CapFactor_Power_max             0.49015


In [30]:
autocorrel=pd.DataFrame(statools.acf(yearly['Energy'],fft=True,qstat=False,alpha=0.05))
autocorrel

,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,40
0,1,0.488344,0.361536,0.288846,0.280791,0.185372,0.150984,0.103881,0.180448,0.115152,...,-0.178255,-0.0835503,-0.0180582,0.0627403,0.0285093,-0.0237914,-0.00646557,0.0295434,0.0114398,0.0520336
1,"[1.0, 1.0]","[0.28510482876486326, 0.6915824238407444]","[0.11453981394195609, 0.6085328554670298]","[0.020880635898701372, 0.5568111386257153]","[0.0002599771537862283, 0.5613227450590462]","[-0.10653822924114706, 0.4772812696698323]","[-0.14574796382450475, 0.44771666220367945]","[-0.19600829020000476, 0.4037693744456652]","[-0.12092329309247682, 0.48181974281332574]","[-0.190649593268081, 0.4209540959160642]",...,"[-0.5652122159303571, 0.20870309307250032]","[-0.4738849962887776, 0.3067844536539582]","[-0.4091309397860464, 0.37301452729998097]","[-0.32836691137375884, 0.4538474393043057]","[-0.3630133794741566, 0.4200319874131315]","[-0.4153998452771418, 0.36781700101951753]","[-0.3981336925859616, 0.38520255222092686]","[-0.36212908915197944, 0.4212159729671288]","[-0.3803248019842708, 0.4032043336359857]","[-0.3397447546733756, 0.44381197685532947]"


In [87]:
lowheadshutdowns=daily.loc[daily['LowHeadShutoff']==1].groupby('WaterYear').count()['Flow'].to_frame().rename(columns={"Flow":"Days"})

lowheadshutdowns['StartDate']=daily[['WaterYear','Flow']].loc[daily['LowHeadShutoff']==1].reset_index().groupby('WaterYear').min()['Date']
lowheadshutdowns['EndDate']=daily[['WaterYear','Flow']].loc[daily['LowHeadShutoff']==1].reset_index().groupby('WaterYear').max()['Date']
lowheadshutdowns['MonthId']=daily[['WaterYear','MonthId']].loc[daily['LowHeadShutoff']==1].reset_index().groupby('WaterYear').min()['MonthId']

#daily.loc[daily['LowHeadShutoff']==1].groupby('WaterYear').min()#['Date']
lowheadshutdowns['Month_power_max']=monthly.loc[lowheadshutdowns['MonthId']][['WaterYear','Power_max']].set_index('WaterYear')
lowheadshutdowns

,Days,StartDate,EndDate,MonthId,Month_power_max
WaterYear,,,,,
1933,3,1934-03-26,1934-03-28,1934.03,180.964667
1947,31,1948-03-04,1948-04-03,1948.03,106.174038
1951,19,1952-03-02,1952-03-20,1952.03,104.964243
1952,9,1953-03-21,1953-03-29,1953.03,187.487855
1955,12,1956-05-05,1956-05-16,1956.05,115.164735
1956,27,1957-03-02,1957-03-28,1957.03,105.902482
1957,60,1958-02-03,1958-04-03,1958.02,110.164461
1960,22,1961-04-02,1961-04-23,1961.04,100.599028
1961,6,1962-04-05,1962-04-12,1962.04,104.188853


In [92]:
lowflowshutdowns=daily.loc[daily['LowFlowShutoff']==1].groupby('Year').count()['Flow'].to_frame().rename(columns={"Flow":"Days"})

lowflowshutdowns['StartDate']=daily[['Year','Flow']].loc[daily['LowFlowShutoff']==1].reset_index().groupby('Year').min()['Date']
lowflowshutdowns['EndDate']=daily[['Year','Flow']].loc[daily['LowFlowShutoff']==1].reset_index().groupby('Year').max()['Date']
lowflowshutdowns['MonthId']=daily[['Year','MonthId']].loc[daily['LowFlowShutoff']==1].reset_index().groupby('Year').min()['MonthId']


lowflowshutdowns['Month_power_max']=monthly.loc[lowflowshutdowns['MonthId']][['Year','Power_max']].set_index('Year')
lowflowshutdowns

,Days,StartDate,EndDate,MonthId,Month_power_max
Year,,,,,
1924,10,1924-10-01,1924-10-10,1924.1,15.370217


## Save Files

In [ ]:
outp=output_data+out_prefix

if dryrun==False:
    daily.to_csv(outp+'_pe_daily.csv')
    monthly.to_csv(outp+'_pe_monthly.csv')
    yearly.to_csv(outp+'_pe_yearly.csv')
    edc.to_csv(outp+'_pe_percentiles.csv')
    monthly_day.to_csv(outp+'_pe_monthly_day.csv')
    daily_slim.to_csv(outp+'_pe_daily_slim.csv')
    weekly.to_csv(outp+'_pe_weekly.csv')
    weekly_slim.to_csv(outp+'_pe_weekly_slim.csv')
    calmonthly.to_csv(outp+'_pe_calmonthly.csv')
    power_exceed.to_csv(outp+'_pe_power_exceed.csv')
    fdc.to_csv(outp+'_pe_fdc.csv')
    autocorrel.to_csv(outp+'_pe_autocorrel.csv')
    #models.to_csv(input_data + "models.csv")
    models.to_csv(input_data + "models.csv")
    lowheadshutdowns.to_csv(outp + "lowheadshutdowns.csv")
    lowflowshutdowns.to_csv(outp + "lowflowshutdowns.csv")

In [96]:
years=daily['WaterYear'].unique().tolist()

for year in years:
    days=daily.loc[daily.WaterYear==year]
    days.to_csv(output_data + 'years/' + out_prefix + '_pe_daily_' + str(year) + '.csv')
 